In [1]:
#%load_ext autoreload
#%autoreload 2

import os
import numpy as np 
from multiprocessing import Pool
import time 
import random
import IPython.display as ipd
import pyaudio
import sys
import librosa
import librosa.display
from tqdm import tqdm
from matplotlib import pyplot as plt
sys.path.append('../../')

import pdb
#import utils.audio as a
#import utils.plots as plots
import utils as utils

/opt/anaconda3/envs/aip37/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Load audio files into dataset

In [2]:
# load (fe)male audio sequences without alpha, beta 
PATH = '../../datasets/freesound.org/'
SR = 22050 

dataset = []
chunk_size = int(SR * .5) # 500ms
num_feats = 80
files = os.listdir(PATH + "wav_denoised/")

for f in tqdm(files):
    if f.split('.')[-1] != 'wav':
        continue
    #pdb.set_trace()
    audio, sr = utils.audio.loadAudio(PATH + "wav/" + f, sr=SR)
    chunks = utils.misc.slidingWindow(audio, chunk_size, chunk_size)
    for i, (chunk) in enumerate(chunks):
        #pdb.set_trace()
        np.save(PATH + "mfcc/" + f.split('.')[0] + "_" + str(i) + "_500ms.npy", librosa.feature.mfcc(y=chunks[i, :], sr=SR, n_mfcc=num_feats))

100%|██████████| 384/384 [00:16<00:00, 23.92it/s]


### Create Feature Vector for all chunks

In [3]:
# load (fe)male audio sequences without alpha, beta 
PATH = '../../datasets/freesound.org/'
SR = 22050 

dataset = []
chunk_size = int(SR * .2) # 500ms
num_feats = 6
files = os.listdir(PATH + "wav_denoised/")
X = []
for f in tqdm(files):
    if f.split('.')[-1] != 'wav':
        continue
    #pdb.set_trace()
    audio, sr = utils.audio.loadAudio(PATH + "wav/" + f, sr=SR)
    chunks = utils.misc.slidingWindow(audio, chunk_size, chunk_size)
    for i, (chunk) in enumerate(chunks):

         # mfcc
        feat = librosa.feature.mfcc(y=chunks[i, :], sr=SR, n_mfcc=num_feats).flatten() # 13x44

        # RMS mag spec
        rms = librosa.feature.rms(y=chunks[i, :]).flatten() #1x44

        # Spectral centroid
        cent = librosa.feature.spectral_centroid(y=chunks[i, :], sr=SR).flatten()#1x44

        # Spectral bandwidth
        band = librosa.feature.spectral_bandwidth(y=chunks[i, :], sr=SR).flatten()#1x44

        # Spectral flatness
        flat = librosa.feature.spectral_flatness(y=chunks[i, :]).flatten()#1x44

        # spectral rolloff
        rolloff = librosa.feature.spectral_rolloff(y=chunks[i, :], sr=SR).flatten()#1x44

        # zero crossing rate 
        zcross = librosa.feature.zero_crossing_rate(chunks[i, :]).flatten()#1x44

        # Spectral contrast
        S = np.abs(librosa.stft(chunks[i, :]))
        spec_contrast = librosa.feature.spectral_contrast(S=S, sr=SR).flatten() #7x44
        
        # concetante to feature vector
        X = np.concatenate((feat, rms, cent, band, flat, rolloff, zcross, spec_contrast))
                            
        #print(np.asarray(X).reshape(-1).shape)
        #pdb.set_trace()
        np.save(PATH + "features/" + f.split('.')[0] + "_" + str(i) + "_200ms.npy", np.asarray(X).reshape(-1))

100%|██████████| 384/384 [01:33<00:00,  4.09it/s]


In [18]:
X.shape

AttributeError: 'list' object has no attribute 'shape'